In [1]:
import random
import shutil
import os
import json
from PIL import Image
import imagesize
random.seed(42)

In [2]:
# Remember to modify paths for your system
og_train_path = "D:/TIL Data/til2020/train/train"
og_annotations_path = "D:/TIL Data/til2020/train.json"

og_annotations = {}
with open(og_annotations_path, 'r') as f:
     og_annotations = json.load(f)

#print(og_annotations)
print(og_annotations['annotations'][0])
#Load 2000 dresses, all of rest

{'area': 2304645, 'iscrowd': 0, 'id': 1, 'image_id': 10, 'category_id': 4, 'bbox': [704, 620, 1401, 1645]}


In [3]:
modanet_train_path = "D:/TIL Data/modanet/train"
modanet_annotations_path = "D:/TIL Data/modanet/modanet2018_instances_train.json"

modanet_annotations = {}
with open(modanet_annotations_path, 'r') as f:
     modanet_annotations = json.load(f)
print(modanet_annotations['images'][0])

{'file_name': '0736791.jpg', 'width': 400, 'id': 736791, 'license': 3, 'height': 600}


In [4]:
merged_path = "D:/TIL Data/Merged"
merged_annotations = {}
merged_annotations['annotations'] = []
merged_annotations['categories'] = og_annotations['categories']
merged_annotations['images'] = []
print(merged_annotations)
merged_ids = 0
annotation_ids=0

{'annotations': [], 'categories': [{'id': 1, 'name': 'tops'}, {'id': 2, 'name': 'trousers'}, {'id': 3, 'name': 'outerwear'}, {'id': 4, 'name': 'dresses'}, {'id': 5, 'name': 'skirts'}], 'images': []}


In [5]:
def copyover(par_dir, file_id):
    file_id = str(file_id) + ".jpg"
    old_file_path = os.path.join(par_dir, file_id)
    new_file_path = os.path.join(merged_path, str(merged_ids)) + ".jpg"
    shutil.copy(old_file_path, new_file_path)

#copyover(og_train_path, 10)

In [6]:
random.seed(42)
og_image_ids = []
for entry in os.scandir(og_train_path):
    if (entry.path.endswith(".jpg")):
        og_string = os.path.basename(entry)
        og_image_ids.append(int(og_string[:-4]))
og_image_ids = list(dict.fromkeys(og_image_ids))
random.shuffle(og_image_ids)
# print(sorted(og_image_ids))

classfreq = {1:0, 2:0, 3:0, 4:0, 5:0}
classlims = {1:2000, 2:2000, 3:2000, 4:3000, 5:2000}

image_to_annotation = {}

for annotation in og_annotations['annotations']:
    if annotation["image_id"] not in og_image_ids:
        continue
    if annotation["image_id"] not in image_to_annotation:
        image_to_annotation[annotation["image_id"]] = []
    image_to_annotation[annotation["image_id"]].append(annotation)
        
merged_ids = 0
annotation_ids = 0
for image_id in og_image_ids:
    cur_classfreq = {1:0, 2:0, 3:0, 4:0, 5:0}
    new_an_list = []
    if image_id not in image_to_annotation:
        continue
    for annotation in image_to_annotation[image_id]:
        if annotation["image_id"] == image_id:
            cur_classfreq[annotation['category_id']]+=1
            new_an_list.append(annotation)
    flag = False
    if (classfreq[4] < classlims[4]): flag = True
    for cat in [1,2,3,5]:
        if cur_classfreq[cat]>0: flag = True
    if flag == True:
        merged_ids += 1
        # print(merged_ids)
        for an in new_an_list:
            annotation_ids += 1
            an['image_id'] = merged_ids
            an['id'] = annotation_ids
            merged_annotations["annotations"].append(an)
        imagedict = {}
        imagedict['file_name'] = str(merged_ids) + '.jpg'
        imagedict['id'] = merged_ids
        imagedict['width'], imagedict['height'] = imagesize.get(os.path.join(og_train_path, str(image_id) + '.jpg'))
        
        merged_annotations["images"].append(imagedict)
        for cat in range(1,6): classfreq[cat] += cur_classfreq[cat]
        #copyover(og_train_path, image_id)
    

In [7]:
print(classfreq)

{1: 945, 2: 1671, 3: 1486, 4: 3604, 5: 1630}


In [8]:
print(merged_annotations["annotations"][0])

{'image_id': 1, 'id': 1, 'iscrowd': 0, 'category_id': 4, 'bbox': [307.0, 732.0, 793.0, 1205.0], 'area': 955565.0}


In [9]:
print(og_annotations['categories'])

[{'id': 1, 'name': 'tops'}, {'id': 2, 'name': 'trousers'}, {'id': 3, 'name': 'outerwear'}, {'id': 4, 'name': 'dresses'}, {'id': 5, 'name': 'skirts'}]


In [10]:
print(modanet_annotations['categories'])

[{'supercategory': 'fashion', 'id': 1, 'name': 'bag'}, {'supercategory': 'fashion', 'id': 2, 'name': 'belt'}, {'supercategory': 'fashion', 'id': 3, 'name': 'boots'}, {'supercategory': 'fashion', 'id': 4, 'name': 'footwear'}, {'supercategory': 'fashion', 'id': 5, 'name': 'outer'}, {'supercategory': 'fashion', 'id': 6, 'name': 'dress'}, {'supercategory': 'fashion', 'id': 7, 'name': 'sunglasses'}, {'supercategory': 'fashion', 'id': 8, 'name': 'pants'}, {'supercategory': 'fashion', 'id': 9, 'name': 'top'}, {'supercategory': 'fashion', 'id': 10, 'name': 'shorts'}, {'supercategory': 'fashion', 'id': 11, 'name': 'skirt'}, {'supercategory': 'fashion', 'id': 12, 'name': 'headwear'}, {'supercategory': 'fashion', 'id': 13, 'name': 'scarf/tie'}]


In [11]:
random.seed(42)
modanet_image_ids = []
for entry in os.scandir(modanet_train_path):
    if (entry.path.endswith(".jpg")):
        og_string = os.path.basename(entry)
        modanet_image_ids.append(int(og_string[:-4]))
        
modanet_image_ids = list(dict.fromkeys(modanet_image_ids))
random.shuffle(modanet_image_ids)
modanet_image_ids = modanet_image_ids[:2500]

classconv = {9:1, 8:2, 5:3, 6:4, 11:5}

image_to_annotation = {}

for annotation in modanet_annotations['annotations']:
    if annotation["image_id"] not in modanet_image_ids:
        continue
    if annotation["image_id"] not in image_to_annotation:
        image_to_annotation[annotation["image_id"]] = []
    image_to_annotation[annotation["image_id"]].append(annotation)
    if annotation['category_id'] in classconv:
        classfreq[classconv[annotation['category_id']]]+=1
                


In [12]:
for image_id in modanet_image_ids:
    if image_id not in image_to_annotation:
        continue
    merged_ids += 1
    image_path = str(image_id)
    while len(image_path) < 7:
        image_path = "0" + image_path
        
    imagedict = {}
    imagedict['file_name'] = str(merged_ids) + '.jpg'
    imagedict['id'] = merged_ids
    imagedict['width'], imagedict['height'] = imagesize.get(os.path.join(modanet_train_path, str(image_path) + ".jpg"))
    
    merged_annotations["images"].append(imagedict)
    
    for an in image_to_annotation[image_id]:
        if an["category_id"] not in classconv:
            continue
        annotation_ids += 1
        an['id'] = annotation_ids
        an["category_id"] = classconv[an["category_id"]]
        an["image_id"] = merged_ids
        merged_annotations["annotations"].append(an)
    
    #copyover(modanet_train_path, image_path)

In [13]:
print(classfreq)
print(len(merged_annotations["images"]))

{1: 2729, 2: 2749, 3: 2730, 4: 4244, 5: 2310}
7679


In [14]:
with open('result.json', 'w') as fp:
    json.dump(merged_annotations, fp)

In [15]:
print(merged_annotations["annotations"][10000])

{'segmentation': [[176, 287, 177, 318, 195, 315, 228, 303, 223, 285]], 'area': 1716, 'iscrowd': 0, 'image_id': 5488, 'bbox': [176, 285, 52, 33], 'category_id': 1, 'id': 10001}


In [16]:
print(og_annotations["annotations"][0])

{'area': 2304645, 'iscrowd': 0, 'id': 4824, 'image_id': 2960, 'category_id': 4, 'bbox': [704, 620, 1401, 1645]}


In [17]:
print(merged_annotations["images"])

[{'file_name': '1.jpg', 'id': 1, 'width': 1551, 'height': 2600}, {'file_name': '2.jpg', 'id': 2, 'width': 462, 'height': 660}, {'file_name': '3.jpg', 'id': 3, 'width': 1280, 'height': 960}, {'file_name': '4.jpg', 'id': 4, 'width': 1632, 'height': 1632}, {'file_name': '5.jpg', 'id': 5, 'width': 3500, 'height': 2333}, {'file_name': '6.jpg', 'id': 6, 'width': 1000, 'height': 1400}, {'file_name': '7.jpg', 'id': 7, 'width': 1024, 'height': 1024}, {'file_name': '8.jpg', 'id': 8, 'width': 735, 'height': 877}, {'file_name': '9.jpg', 'id': 9, 'width': 1536, 'height': 2048}, {'file_name': '10.jpg', 'id': 10, 'width': 3456, 'height': 5184}, {'file_name': '11.jpg', 'id': 11, 'width': 960, 'height': 720}, {'file_name': '12.jpg', 'id': 12, 'width': 3456, 'height': 5184}, {'file_name': '13.jpg', 'id': 13, 'width': 1936, 'height': 1936}, {'file_name': '14.jpg', 'id': 14, 'width': 3264, 'height': 3264}, {'file_name': '15.jpg', 'id': 15, 'width': 2448, 'height': 3264}, {'file_name': '16.jpg', 'id': 16, 